In [1]:
import pandas as pd
import numpy as np
Path = r'C:\Users\rgupta4\Assignment For DA Candidates\Data Analyst Assignment Part 1\funnel-analysis'

In [2]:
LIST = ['user_table','home_page_table','search_page_table','payment_page_table','payment_confirmation_table']

data = pd.DataFrame
for table in LIST:
    if data.empty:
        data = pd.read_csv(Path+'\\'+table+'.csv')
    else:
        data = pd.merge(data, pd.read_csv(Path+'\\'+table+'.csv') , on = 'user_id' , how='left', suffixes = ('',table))
        if 'page'+table in data.columns:
            #Updating known and unown values with 1 and 0 respectively
            data['page'+table] = np.where(pd.isnull(data['page'+table]),0,1)
            #Rename column
            data.rename(columns = {'page'+table: table}, inplace = True)
        elif 'page' in data.columns:
            data['page'] = np.where(pd.isnull(data['page']),0,1)
            
data.rename(columns = {'page': 'home_page_table'}, inplace = True)

In [3]:
data.head()

,user_id,date,device,sex,home_page_table,search_page_table,payment_page_table,payment_confirmation_table
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [4]:
data_summary_sex = pd.DataFrame
for table in LIST:
    if table == 'user_table':
        continue
    elif table=='home_page_table':
        data_summary_sex = data.groupby(['sex'])['home_page_table'].agg(sum)*100/len(data)
#         data_summary.reset_index(inplace=True)
    else:
        data_summary_sex = pd.merge(data_summary_sex, data.groupby(['sex'])[table].agg(sum)*100/len(data), \
                                    suffixes = ('',table), left_on = 'sex' , right_on = 'sex')

data_summary_sex.reset_index(inplace=True)
data_summary_sex['Conversion'] = data_summary_sex['payment_confirmation_table']*100/ data_summary_sex['home_page_table']

In [5]:
data_summary_device = pd.DataFrame
for table in LIST:
    if table == 'user_table':
        continue
    elif table=='home_page_table':
        data_summary_device = data.groupby(['device'])['home_page_table'].agg(sum)*100/len(data)
#         data_summary.reset_index(inplace=True)
    else:
        data_summary_device = pd.merge(data_summary_device, data.groupby(['device'])[table].agg(sum)*100/len(data), \
                                    suffixes = ('',table), left_on = 'device' , right_on = 'device')

        
data_summary_device.reset_index(inplace=True)
data_summary_device['Conversion'] = data_summary_device['payment_confirmation_table']*100/ data_summary_device['home_page_table']

In [6]:
data_summary_sex

,sex,home_page_table,search_page_table,payment_page_table,payment_confirmation_table,Conversion
0,Female,49.861726,25.084071,3.429204,0.266593,0.534664
1,Male,50.138274,24.915929,3.241150,0.233407,0.465527


In [7]:
data_summary_device

,device,home_page_table,search_page_table,payment_page_table,payment_confirmation_table,Conversion
0,Desktop,66.59292,33.29646,3.329646,0.165929,0.249169
1,Mobile,33.40708,16.70354,3.340708,0.334071,1.000000


In [8]:
##1-- full picture of comparision of funnel conversion rate for both desktop and mobile

#By analyzing the data we conclude that although Desktop is preferred mode for visiting the site (double the Mobile).
#However, the conversion rate (from home page to payment confirmation page) is significantly low 1/4 of Mobile use.
#There is not much difference in gender participation

##2- Insights on what the product team should focus on in order to improve conversion rate 

#the major drops between pages is seen from search page to payment page especially for desktop devices . Also,
#from payment-page to Payment_confirmation page (~10% retention)

#My 2 recommedetion to product team would be 
#A) To optimize search results page so that people are keen to buy more products. Probably could do A/B testing.
#B) To review the payment page, if there is any issue with the payment selection. Is there any payment failure, 
#    is the popular payment mode listed in the selection box, is COD mode available. etc